Установим необходимые библиотеки для парсинга дискрипторов из библиотеки rdkit.

In [ ]:
!pip install rdkit-pypi
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import pandas as pd
import numpy as np


С помощью библиотеки psndas считываем датасет.

In [12]:
data = pd.read_excel('7_whit_cid.xlsx')


In [34]:
#Выбираем колонку по которой будем парсить дискрипторы
smiles = data['CanonicalSMILES']
#Чтобы сократить время расчетов временно удалим все дублирующиеся smiles
smiles = smiles.drop_duplicates()
smiles = smiles.reset_index(drop = True)

In [35]:
#Удалим строчку с пустым занчением чтобы парсер не выдавал ошибку
smiles = smiles.drop(labels = [5],axis = 0)

Проверим корекктность полученого named series

In [ ]:
smiles

In [ ]:
#Создадим пустой датафрейм куда будем добавлять значения дескрипторов
df = pd.DataFrame()
for i in smiles:  
  #Считываем молекулу из smiles формата
  mol = Chem.MolFromSmiles(i)
  #Создадим переменную в которую запишем расчет всех дескрипторов доступных в rdkit
  calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
  #Запишем в переменную наименование всех дескрипторов
  desc_names = calc.GetDescriptorNames()
  #Создадим пустой список в который будут записываться значения дескрипторов
  Mol_descriptors = []
  #Добавим водороды к молекуле
  mol = Chem.AddHs(mol)
  #НАходим значения всех дескрипторов для молекулы
  descriptors = calc.CalcDescriptors(mol)
  #Заносим полученные значения в список
  Mol_descriptors.append(descriptors)
  #Формируем дата фрейм где назавнием колонок будут наименования дескрипторов а первый ряд это значения соответсвующих десркипторов
  a = pd.DataFrame(Mol_descriptors, columns=desc_names)
  #Добавляем полученную строку в изначальный датафрейм
  df = pd.concat([df,a])

Проверим полученный датафрейм

In [ ]:
df

Считываем изначальную колонку CanonicalSMILES

In [62]:
smil = data['CanonicalSMILES']

При помощи функции merge востанавливаваем убранные ранее дубликаты. Для это используем left.join а индексной колонкой делаем общую колонку 'CanonicalSMILES

In [64]:
rdkit_desk = pd.merge(smil, df, how='left', on='CanonicalSMILES')


Выгружаем полученный датасет, проверяем его и добавляем дескрипторы в изначальную

In [65]:
rdkit_desk.to_excel('rd_desk.xlsx', index = False)